Lambda School Data Science

*Unit 2, Sprint 2, Module 3*

---

# Cross-Validation


## Assignment
- [x] [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2), then submit your dataset.
- [x] Continue to participate in our Kaggle challenge. 
- [ ] Use scikit-learn for hyperparameter optimization with RandomizedSearchCV.
- [ ] Submit your predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file. Or you can use the Kaggle API to submit your predictions.)
- [x] Commit your notebook to your fork of the GitHub repo.


You won't be able to just copy from the lesson notebook to this assignment.

- Because the lesson was ***regression***, but the assignment is ***classification.***
- Because the lesson used [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html), which doesn't work as-is for _multi-class_ classification.

So you will have to adapt the example, which is good real-world practice.

1. Use a model for classification, such as [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)
2. Use hyperparameters that match the classifier, such as `randomforestclassifier__ ...`
3. Use a metric for classification, such as [`scoring='accuracy'`](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values)
4. If you’re doing a multi-class classification problem — such as whether a waterpump is functional, functional needs repair, or nonfunctional — then use a categorical encoding that works for multi-class classification, such as [OrdinalEncoder](https://contrib.scikit-learn.org/categorical-encoding/ordinal.html) (not [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html))



## Stretch Goals

### Reading
- Jake VanderPlas, [Python Data Science Handbook, Chapter 5.3](https://jakevdp.github.io/PythonDataScienceHandbook/05.03-hyperparameters-and-model-validation.html), Hyperparameters and Model Validation
- Jake VanderPlas, [Statistics for Hackers](https://speakerdeck.com/jakevdp/statistics-for-hackers?slide=107)
- Ron Zacharski, [A Programmer's Guide to Data Mining, Chapter 5](http://guidetodatamining.com/chapter5/), 10-fold cross validation
- Sebastian Raschka, [A Basic Pipeline and Grid Search Setup](https://github.com/rasbt/python-machine-learning-book/blob/master/code/bonus/svm_iris_pipeline_and_gridsearch.ipynb)
- Peter Worcester, [A Comparison of Grid Search and Randomized Search Using Scikit Learn](https://blog.usejournal.com/a-comparison-of-grid-search-and-randomized-search-using-scikit-learn-29823179bc85)

### Doing
- Add your own stretch goals!
- Try other [categorical encodings](https://contrib.scikit-learn.org/categorical-encoding/). See the previous assignment notebook for details.
- In additon to `RandomizedSearchCV`, scikit-learn has [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Another library called scikit-optimize has [`BayesSearchCV`](https://scikit-optimize.github.io/notebooks/sklearn-gridsearchcv-replacement.html). Experiment with these alternatives.
- _[Introduction to Machine Learning with Python](http://shop.oreilly.com/product/0636920030515.do)_ discusses options for "Grid-Searching Which Model To Use" in Chapter 6:

> You can even go further in combining GridSearchCV and Pipeline: it is also possible to search over the actual steps being performed in the pipeline (say whether to use StandardScaler or MinMaxScaler). This leads to an even bigger search space and should be considered carefully. Trying all possible solutions is usually not a viable machine learning strategy. However, here is an example comparing a RandomForestClassifier and an SVC ...

The example is shown in [the accompanying notebook](https://github.com/amueller/introduction_to_ml_with_python/blob/master/06-algorithm-chains-and-pipelines.ipynb), code cells 35-37. Could you apply this concept to your own pipelines?


### BONUS: Stacking!

Here's some code you can use to "stack" multiple submissions, which is another form of ensembling:

```python
import pandas as pd

# Filenames of your submissions you want to ensemble
files = ['submission-01.csv', 'submission-02.csv', 'submission-03.csv']

target = 'status_group'
submissions = (pd.read_csv(file)[[target]] for file in files)
ensemble = pd.concat(submissions, axis='columns')
majority_vote = ensemble.mode(axis='columns')[0]

sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission[target] = majority_vote
submission.to_csv('my-ultimate-ensemble-submission.csv', index=False)
```

In [1]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [49]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

pd.set_option('display.max_columns', 999)

# Merge train_features.csv & train_labels.csv
train = pd.merge(pd.read_csv(DATA_PATH+'waterpumps/train_features.csv'), 
                 pd.read_csv(DATA_PATH+'waterpumps/train_labels.csv'))

# Read test_features.csv & sample_submission.csv
test = pd.read_csv(DATA_PATH+'waterpumps/test_features.csv')
sample_submission = pd.read_csv(DATA_PATH+'waterpumps/sample_submission.csv')

In [50]:
train, val = train_test_split(train, random_state=0)

In [51]:
def wrangle(data):
    df = data.copy()
    
    weird_mismatches = {
        5: ('Tanga', 4),
        11: ('Shinyanga', 17),
        14: ('Shinyanga', 17),
        17: ('Mwanza', 19),
        18: ('Lindi', 8),
        24: ('Arusha', 2),
        40: ('Pwani', 6),
        60: ('Pwani', 6),
        80: ('Lindi', 8),
        90: ('Mtwara', 9),
        99: ('Mtwara', 9)
    }
    for code in weird_mismatches:
        wrong_region, right_code = weird_mismatches[code]
        df.loc[(df.region_code == code) & (df.region == wrong_region), 'region_code'] = right_code
    
    # About 3% of the time, latitude has small values near zero,
    # outside Tanzania, so we'll treat these values like zero.
    df['latitude'] = df['latitude'].replace(-2e-08, 0)
    
    # When columns have zeros and shouldn't, they are like null values.
    # So we will replace the zeros with nulls, and impute missing values later.
    # Also create a "missing indicator" column, because the fact that
    # values are missing may be a predictive signal.
    cols_with_zeros = ['longitude', 'latitude', 'construction_year', 
                       'gps_height', 'population', 'amount_tsh']
    for col in cols_with_zeros:
        df[col] = df[col].replace(0, np.nan)
    
    # Convert date_recorded to datetime
    df['date_recorded'] = pd.to_datetime(df['date_recorded'], infer_datetime_format=True)
    
    # Extract components from date_recorded, then drop the original column
    df['year_recorded'] = df['date_recorded'].dt.year
    df['month_recorded'] = df['date_recorded'].dt.month
    df['day_recorded'] = df['date_recorded'].dt.day
    df = df.drop(columns='date_recorded')
    
    # Engineer feature: how many years from construction_year to date_recorded
    df['years'] = df['year_recorded'] - df['construction_year']
    
    # return the wrangled dataframe
    
    evil_dimensions = ['extraction_type_group', 'management', 'water_quality',
                       'payment', 'extraction_type', 'waterpoint_type_group',
                       'scheme_management', 'quantity_group', 'source',
                       'source_class', 'recorded_by', 'region']
        
    df = df.drop(evil_dimensions, axis=1)
    return df

train = wrangle(train)
val = wrangle(val)
test = wrangle(test)

In [52]:
# The status_group column is the target
target = 'status_group'

# Get a dataframe with all train columns except the target
# I need to keep the id column from the test set apparently
train_features = train.drop(columns=[target, 'id'])

# Get a list of the numeric features
numeric_features = train_features.select_dtypes(include='number').columns.tolist()

# Get a series with the cardinality of the nonnumeric features
cardinality = train_features.select_dtypes(exclude='number').nunique()

# Get a list of all categorical features with cardinality <= 50
categorical_features = cardinality[cardinality <= 50].index.tolist()

# Combine the lists 
features = numeric_features + categorical_features

# Arrange data into X features matrix and y target vector 
X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]

In [63]:
from sklearn.feature_selection import f_regression, chi2, SelectKBest, f_classif
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Ridge
from category_encoders import OneHotEncoder, OrdinalEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import StandardScaler

In [54]:
%%time

pipeline = make_pipeline(
    OrdinalEncoder(),
    IterativeImputer(random_state=0, imputation_order='descending'),
    StandardScaler(),
    RandomForestClassifier(random_state=0,
                           n_jobs=-1,
                           n_estimators=100)
)
pipeline.fit(X_train, y_train)
print('Validation Accuracy', pipeline.score(X_val, y_val))

Validation Accuracy 0.8023569023569024
Wall time: 4.42 s


In [55]:
%%time
%matplotlib inline
import matplotlib.pyplot as plt
pipeline = make_pipeline(
    OrdinalEncoder(),
    IterativeImputer(random_state=0, imputation_order='descending'),
    StandardScaler(), 
    SelectKBest(f_regression, k=20),
    RandomForestClassifier(random_state=0,
                           n_jobs=-1)
)

Wall time: 30 ms


In [80]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

pipeline = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(), 
    StandardScaler(), 
    SelectKBest(f_classif),
    RandomForestClassifier(random_state=0,
                           n_jobs=-1)
)

param_distributions = {
    'simpleimputer__strategy': ['mean', 'median'], 
    'selectkbest__k': range(1, len(X_train.columns)+1), 
    'randomforestclassifier__n_estimators': range(50, 160, 10)
}

# If you're on Colab, decrease n_iter & cv parameters
search = RandomizedSearchCV(
    pipeline, 
    param_distributions=param_distributions, 
    n_iter=100, 
    cv=5, 
    scoring='precision_weighted', 
    verbose=10, 
    return_train_score=True, 
    n_jobs=-1
)

search.fit(X_train, y_train);

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   12.1s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   21.8s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   25.2s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   50.3s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done  69 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done  82 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done  97 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done 129 tasks      | elapsed:  3.7min
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-1)]: Done 165 tasks      | elapsed:  4.9min
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  5

In [81]:
print('Best hyperparameters', search.best_params_)
print('Cross-validation MAE', -search.best_score_)

Best hyperparameters {'simpleimputer__strategy': 'median', 'selectkbest__k': 23, 'randomforestclassifier__n_estimators': 140}
Cross-validation MAE -0.7931999272658178


In [82]:
pd.DataFrame(search.cv_results_).sort_values(by='rank_test_score').T

,39,6,53,46,14,82,47,87,20,90,56,83,42,98,30,86,89,32,10,66,55,15,97,85,65,2,73,70,79,44,50,29,77,16,80,13,33,26,69,19,31,4,93,8,21,48,81,92,3,71,7,68,74,5,95,52,23,51,24,94,99,91,88,18,45,27,36,96,38,40,63,57,41,76,12,72,35,62,22,49,0,43,17,58,34,37,1,25,78,64,60,84,75,28,11,9,61,54,59,67
mean_fit_time,13.1384,13.0092,12.6373,9.49504,8.36448,7.75232,13.0239,17.0656,6.2231,5.49927,12.2536,14.1596,8.51076,10.6122,15.2592,7.68744,10.2927,6.02377,13.3116,10.9143,13.1008,11.6984,8.24663,11.4899,14.8317,6.75461,5.90757,15.9626,11.0168,15.4061,9.56503,10.7249,10.0152,11.2857,12.3168,4.28869,9.97248,7.58824,6.02504,3.95659,14.488,9.49824,8.9203,7.56156,7.19821,11.727,7.17782,4.01375,7.43467,6.70188,12.3002,4.49261,7.2554,11.0317,8.1379,11.1583,4.73347,13.9457,7.50473,9.37326,8.18113,4.72804,9.13662,3.77168,8.4794,4.90731,7.09142,6.29483,4.23943,6.80412,4.67577,5.89751,6.71948,4.67521,4.19391,5.20336,4.64903,3.2899,3.13814,5.5572,4.66137,6.74214,4.67348,5.74986,4.33287,2.20118,2.22079,3.18787,3.14518,3.25518,2.45262,3.76113,4.52096,3.24311,4.84446,4.40559,2.82996,6.42867,6.31276,3.61408
std_fit_time,1.35857,1.98655,2.75437,1.07648,1.07374,1.42882,3.41319,0.54152,0.619676,1.01329,2.70542,1.35476,0.769371,1.80578,1.16937,2.88365,0.414358,1.97292,1.06555,4.14585,1.58092,0.910903,1.73841,2.04828,1.77296,0.968622,0.392492,0.405471,3.02761,0.711891,1.3213,2.1036,2.38673,1.20067,2.15358,0.387152,1.3691,1.45771,1.31031,0.404628,1.38047,2.68211,0.762427,0.884478,0.389218,3.83212,0.180631,0.817704,2.40789,2.6513,1.51841,1.49056,1.01167,1.82315,1.90746,2.54577,0.948923,0.424566,0.326824,0.566057,0.880979,1.36804,3.97985,0.777909,3.36996,1.19215,0.474977,2.2774,0.596765,1.79429,0.506917,0.999731,0.486263,1.35228,0.666917,0.530415,1.06694,0.686232,0.39026,0.67443,0.84191,2.26957,0.230927,1.24355,0.95963,0.286008,0.464988,1.71694,0.269763,0.69403,0.72617,0.759572,0.774672,0.303626,1.43707,1.53018,0.984137,0.477486,0.349721,0.587167
mean_score_time,2.88505,3.14349,1.32464,1.55624,2.87468,2.93122,2.74162,1.3192,1.7289,1.64824,2.21389,2.99592,1.56157,1.88756,3.28512,3.22972,1.73329,2.9517,1.9407,1.66698,2.52087,3.3406,3.23967,3.30192,2.53143,2.05095,2.05122,1.8037,3.42279,1.17089,3.56188,3.96157,2.2205,2.51434,2.40154,1.45965,1.67569,2.3271,2.15938,1.95795,3.15683,1.18761,2.47805,1.77103,2.0546,2.10294,2.02529,2.35405,2.61375,2.71926,2.28351,2.57911,1.1336,3.70992,1.84979,4.24105,1.80433,2.19961,1.5924,2.60929,2.69291,0.881514,2.11577,1.05857,3.22685,1.20217,1.50837,1.34861,1.61429,2.68874,1.24295,2.09211,2.1062,3.35255,1.44396,1.37152,1.56289,1.259,2.09173,2.24858,1.11333,1.74748,1.39202,1.78133,1.07878,0.556488,2.2434,1.06487,1.24776,0.827169,1.11339,0.884515,1.37182,1.1869,2.38545,1.39742,0.81314,2.09551,1.05448,1.98887
std_score_time,1.17146,1.33335,0.268519,0.984605,0.451517,1.27774,1.99079,0.575138,0.848184,0.600696,0.543364,0.887371,0.804891,0.625279,1.06385,1.64534,0.458101,0.512008,0.559846,0.217061,0.98283,0.771847,1.07022,0.977728,1.51731,0.701108,0.880118,0.396711,0.663228,0.676872,1.78598,0.80092,0.706997,0.484534,0.281968,0.655465,0.909187,0.998399,1.60881,0.225401,1.12714,0.528537,0.695522,0.94606,0.72567,0.453158,0.459823,0.374385,1.61293,1.71923,1.43207,0.956652,0.488985,1.51378,1.50368,1.56197,1.17602,0.52562,0.530953,0.386358,1.37851,0.185821,1.88454,0.17534,1.33587,0.380086,0.286007,0.667272,0.219146,0.622004,0.602348,0.957495,0.37473,1.22378,0.274501,0.730416,0.676394,0.489505,0.569088,0.21275,0.769935,1.23176,0.342285,0.732771,0.648236,0.172688,0.363895,0.471877,0.212761,0.332263,0.47176,0.357261,0.419726,0.471141,1.98642,0.398514,0.334061,0.48098,0.466727,0.366213
param_simpleimputer__strategy,median,median,median,median,mean,median,median,mean,mean,mean,mean,mean,mean,median,median,mean,median,mean,median,median,median,median,median,median,mean,mean,median,median,mean,mean,mean,mean,mean,mean,mean,median,mean,median,mean,mean,median,mean,mean,mean,median,mean,mean,median,median,mean,median,mean,mean,mean,median,median,mean,mean

In [83]:
pipeline = search.best_estimator_

print('Validation Accuracy', pipeline.score(X_val, y_val))

Validation Accuracy 0.8027609427609428
